In [1]:
import os
import re
import sys
import toml
import json
import numpy as np
from tqdm import tqdm

In [2]:
root = os.getcwd().split('notebooks')[0]

In [3]:
# Get data and inference ids
inference_ids = ["grwmh_greenshields_model_greenshields_sim_learn_noise_n200",
                  "grwmh_daganzos_model_daganzos_sim_learn_noise_n200",
                  "grwmh_delcastillos_model_delcastillos_sim_learn_noise_n200",
                  "grwmh_greenbergs_model_greenbergs_sim_learn_noise_n200",
                  "grwmh_underwoods_model_underwoods_sim_learn_noise_n200",
                  "grwmh_northwesterns_model_northwesterns_sim_learn_noise_n200",
                  "grwmh_newells_model_newells_sim_learn_noise_n200",
                  "grwmh_wangs_model_wangs_sim_learn_noise_n200",
                  "grwmh_smulders_model_smulders_sim_learn_noise_n200",
                  "grwmh_deromphs_model_deromphs_sim_learn_noise_n200"]
data_ids = ["greenshields_fd_log_simulation_n200",
            "daganzos_fd_log_simulation_n200",
            "delcastillos_fd_log_simulation_n200",
            "greenbergs_fd_log_simulation_n200",
            "underwoods_fd_log_simulation_n200",
            "northwesterns_fd_log_simulation_n200",
            "newells_fd_log_simulation_n200",
            "wangs_fd_log_simulation_n200",
            "smulders_fd_log_simulation_n200",
            "deromphs_fd_log_simulation_n200"]

In [4]:
# Proposal adjustments for mcmc tuning
proposal_adjustments = { 
    "daganzos": {"vanilla_mcmc":0.8,"thermodynamic_integration_mcmc":0.6},
    "delcastillos": {"vanilla_mcmc":0.8,"thermodynamic_integration_mcmc":0.6},
    "deromphs": {"vanilla_mcmc":0.8,"thermodynamic_integration_mcmc":0.5},
    "greenbergs": {"vanilla_mcmc":0.8,"thermodynamic_integration_mcmc":0.6},
    "greenshields": {"vanilla_mcmc":0.8,"thermodynamic_integration_mcmc":0.5},
    "newells": {"vanilla_mcmc":0.8,"thermodynamic_integration_mcmc":0.5},
    "northwesterns": {"vanilla_mcmc":0.8,"thermodynamic_integration_mcmc":0.6},
    "smulders": {"vanilla_mcmc":0.8,"thermodynamic_integration_mcmc":0.6},
    "underwoods": {"vanilla_mcmc":0.8,"thermodynamic_integration_mcmc":0.6},
    "wangs": {"vanilla_mcmc":0.8,"thermodynamic_integration_mcmc":0.6}
}

## Create simulation parameters for new dataset size

In [5]:
# New dataset size
new_n = 500
# Flag for exporting parameters to file
export = False

In [6]:
for data_id in tqdm(data_ids):
    
    # Import toml file
    simulation_filename = os.path.join(root,'data','input','simulation_parameters',(data_id+'.toml'))
    if os.path.exists(simulation_filename):
        simulation_parameters  = toml.load(simulation_filename)
    
    # Ammend dataset size in id and rho_steps
    simulation_parameters['id'] = simulation_parameters['id'].replace("_n200",f"_n{str(new_n)}")
    simulation_parameters['rho']['interval1']['rho_steps'] = int(new_n)
    
    # Get new simulation filename
    new_simulation_filename = simulation_filename.replace("_n200",f"_n{str(new_n)}")

    # Export
    if export:
        with open(new_simulation_filename, 'w') as f:
            new_toml_string = toml.dump(simulation_parameters, f)
    
    

100%|██████████| 10/10 [00:00<00:00, 1058.77it/s]


## Create inference parameters for marginal likelihood comparisons
This refers to creating copies of an inference id for running inference on different datasets

In [7]:
# Pick inference id index
index = 9
# Flag for exporting parameters to file
export = False

# Make sure index is in list
assert index < len(inference_ids)

# Get inference id
inference_id = inference_ids[index]

inference_id

'grwmh_deromphs_model_deromphs_sim_learn_noise_n200'

In [8]:
# Loop through datasets
for data_id in tqdm(data_ids):
        
    # Get dataset FD model name
    data_fd = data_id.split('_fd',1)[0]
    model_fd = inference_id.split('_model',1)[0].split("_",1)[1]
    
#     print('data_fd:',data_fd,'model_fd:',model_fd)
    
    # Replace simulation dataset iff model_fd != data_fd
    if data_fd != model_fd: 
        new_inference_id = re.sub(r'\_model.+\_sim',f'_model_{data_fd}_sim',inference_id)
    else: continue
        
    # Import toml file
    inference_filename = os.path.join(root,'data','input','inference_parameters',model_fd,(inference_id+'.toml'))
    if os.path.exists(inference_filename):
        inference_parameters = toml.load(inference_filename)
    
    # Change id and data fundamental diagram properties
    inference_parameters['id'] = new_inference_id
    inference_parameters['data_id'] = re.sub(r'.+\_fd',f'{data_fd}_fd',inference_parameters['data_id'])
    inference_parameters['data_fundamental_diagram'] = data_fd
        
    # Change proposal adjustments
    inference_parameters['inference']['vanilla_mcmc']['transition_kernel']['adapt_proposal_during_burnin'] = "True"
    inference_parameters['inference']['vanilla_mcmc']['transition_kernel']['proposal_factor_adjustment'] = float(proposal_adjustments[data_fd]['vanilla_mcmc'])
    inference_parameters['inference']['thermodynamic_integration_mcmc']['transition_kernel']['adapt_proposal_during_burnin'] = "True"
    inference_parameters['inference']['thermodynamic_integration_mcmc']['transition_kernel']['proposal_factor_adjustment'] = float(proposal_adjustments[data_fd]['thermodynamic_integration_mcmc'])
    
    print(new_inference_id)
#     print(json.dumps(inference_parameters,indent=2))
    # Export
    if export:
        new_inference_filename = os.path.join(root,'data','input','inference_parameters',model_fd,(new_inference_id+'.toml'))
        with open(new_inference_filename, 'w') as f:
            new_toml_string = toml.dump(inference_parameters, f)
    
#     sys.exit(1)
    

100%|██████████| 10/10 [00:00<00:00, 327.64it/s]

grwmh_deromphs_model_greenshields_sim_learn_noise_n200
grwmh_deromphs_model_daganzos_sim_learn_noise_n200
grwmh_deromphs_model_delcastillos_sim_learn_noise_n200
grwmh_deromphs_model_greenbergs_sim_learn_noise_n200
grwmh_deromphs_model_underwoods_sim_learn_noise_n200
grwmh_deromphs_model_northwesterns_sim_learn_noise_n200
grwmh_deromphs_model_newells_sim_learn_noise_n200
grwmh_deromphs_model_wangs_sim_learn_noise_n200
grwmh_deromphs_model_smulders_sim_learn_noise_n200


## Create inference parameters for prior diffusion sensitivity

In [27]:
# Pick inference id index
index = 7
# Flag for exporting parameters to file
export = True
# String characterising prior diffusion
prior_diffusion = 'informative_prior'
# 'diffuse_prior', 'regular_prior', 'informative_prior'
alpha_steps = {"diffuse_prior":0.1,"regular_prior":0.5,"informative_prior":1}
proposals = {"diffuse_prior":"True","regular_prior":"True","informative_prior":"False"}

# Make sure index is in list
assert index < len(inference_ids)

# Get inference id
inference_id = inference_ids[index]

inference_id

'grwmh_wangs_model_wangs_sim_learn_noise_n200'

In [28]:
# Loop through datasets
for data_id in tqdm(data_ids):
        
    # Get dataset FD model name
    data_fd = data_id.split('_fd',1)[0]
    model_fd = inference_id.split('_model',1)[0].split("_",1)[1]
#     print('data_fd:',data_fd,'model_fd:',model_fd)

    # Replace simulation dataset iff model_fd != data_fd
    inference_id = re.sub(r'\_model.+\_sim',f'_model_{data_fd}_sim',inference_id)
    # Import toml file
    inference_filename = os.path.join(root,'data','input','inference_parameters',model_fd,(inference_id+'.toml'))
    if os.path.exists(inference_filename):
        inference_parameters = toml.load(inference_filename)
        
    # Change inference id
    inference_parameters['id'] = (inference_id+'_'+prior_diffusion)
    # Change other things
    inference_parameters["inference"]['vanilla_mcmc']["transition_kernel"]["adapt_proposal_during_burnin"] = "False"
    del inference_parameters["inference"]['vanilla_mcmc']["transition_kernel"]["beta_step"]
    del inference_parameters["inference"]['thermodynamic_integration_mcmc']["transition_kernel"]["beta_step"]
    inference_parameters["inference"]['thermodynamic_integration_mcmc']["transition_kernel"]["alpha_step"] = alpha_steps[prior_diffusion]
    inference_parameters["inference"]['thermodynamic_integration_mcmc']["transition_kernel"]["dynamic_proposal"] = proposals[prior_diffusion]
    
    p0 = list(inference_parameters['inference']['initialisation']['p0'])
    
    for i,k in enumerate(inference_parameters['inference']['priors']):
        if k == 'sigma2':
            inference_parameters['inference']['priors'][k]['loc'] = np.round(p0[i],2)
        else:
            inference_parameters['inference']['priors'][k]['loc'] = np.round(p0[i],1)
    
#     print(json.dumps(inference_parameters,indent=2))
        
    # Export
    if export:
        new_inference_filename = os.path.join(root,'data','input','inference_parameters',model_fd,prior_diffusion,(inference_id+'_'+prior_diffusion+'.toml'))
        if not os.path.exists(os.path.dirname(new_inference_filename)):
            os.makedirs(os.path.dirname(new_inference_filename))
#         if not os.path.exists(new_inference_filename):
        print(inference_id+'_'+prior_diffusion)
        with open(new_inference_filename, 'w') as f:
            new_toml_string = toml.dump(inference_parameters, f)

100%|██████████| 10/10 [00:00<00:00, 349.67it/s]

grwmh_wangs_model_greenshields_sim_learn_noise_n200_informative_prior
grwmh_wangs_model_daganzos_sim_learn_noise_n200_informative_prior
grwmh_wangs_model_delcastillos_sim_learn_noise_n200_informative_prior
grwmh_wangs_model_greenbergs_sim_learn_noise_n200_informative_prior
grwmh_wangs_model_underwoods_sim_learn_noise_n200_informative_prior
grwmh_wangs_model_northwesterns_sim_learn_noise_n200_informative_prior
grwmh_wangs_model_newells_sim_learn_noise_n200_informative_prior
grwmh_wangs_model_wangs_sim_learn_noise_n200_informative_prior
grwmh_wangs_model_smulders_sim_learn_noise_n200_informative_prior
grwmh_wangs_model_deromphs_sim_learn_noise_n200_informative_prior


## Changing proposals

In [11]:
a = [ 0.02, 0.025, 0.005, 0.025, 0.07]
factor = 5/3
print('['+", ".join([str(factor*e) for e in a])+']')

[0.03333333333333333, 0.04166666666666667, 0.008333333333333333, 0.04166666666666667, 0.11666666666666668]


In [12]:
_n = 30
_pow = 5
np.array([(i/(_n-1))**_pow for i in range(0,_n)]).reshape((_n))

array([0.00000000e+00, 4.87539728e-08, 1.56012713e-06, 1.18472154e-05,
       4.99240681e-05, 1.52356165e-04, 3.79110892e-04, 8.19408020e-04,
       1.59757018e-03, 2.87887334e-03, 4.87539728e-03, 7.85187607e-03,
       1.21315486e-02, 1.81020088e-02, 2.62210567e-02, 3.70225481e-02,
       5.11222458e-02, 6.92236695e-02, 9.21239468e-02, 1.20719663e-01,
       1.56012713e-01, 1.99116149e-01, 2.51260034e-01, 3.13797291e-01,
       3.88209554e-01, 4.76113015e-01, 5.79264282e-01, 6.99566221e-01,
       8.39073813e-01, 1.00000000e+00])

## Deleting empty folders

In [13]:
os.getcwd()

'/Users/Yannis/code/probabilistic_traffic_flow_modelling/notebooks'

In [19]:
# root = '/Users/Yannis/code/probabilistic_traffic_flow_modelling/data/output/experiment_data/'
# folders = list(os.walk(root))[1:]

# for folder in folders:
#     if not folder[2]:
#         os.rmdir(folder[0])